In [ ]:
# Import TensorFlow library, which is essential for building and training deep learning models
import tensorflow as tf

# Import Keras layers and models modules from TensorFlow for building neural networks
from tensorflow.keras import layers, models

# Import ImageDataGenerator class for real-time image augmentation and data preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Import the ResNet50 model from TensorFlow's applications module for transfer learning
from tensorflow.keras.applications import ResNet50

# Import Matplotlib's pyplot for plotting graphs and visualizations (e.g., training curves)
import matplotlib.pyplot as plt

# Import classification metrics from scikit-learn to evaluate the model's performance
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score, ConfusionMatrixDisplay

# Import NumPy for numerical operations like array manipulation
import numpy as np


In [ ]:
# Define the paths for training and validation data
train_data_dir = 'data/train'  # Directory for training images
validation_data_dir = 'data/validation'  # Directory for validation images

In [ ]:
# Create a list of dictionaries to store label information for hand gestures used in the sign language detection model.
# Each dictionary contains a 'name' representing the gesture and an 'id' that serves as a unique identifier for that gesture.
label_info = [
    {'name': 'hello', 'id': 1},       # 'hello' gesture mapped to ID 1
    {'name': 'thanks', 'id': 2},      # 'thanks' gesture mapped to ID 2
    {'name': 'yes', 'id': 3},         # 'yes' gesture mapped to ID 3
    {'name': 'no', 'id': 4},          # 'no' gesture mapped to ID 4
    {'name': 'iloveyou', 'id': 5}     # 'I love you' gesture mapped to ID 5
]


In [ ]:
# Create a dictionary that maps class indices to corresponding label names.
# This is done using a dictionary comprehension that enumerates over the label_info list.
label_dict = {i: label['name'] for i, label in enumerate(label_info)}

# Print the resulting label dictionary to verify the mapping.
print("Label dictionary:", label_dict)  # Output the label dictionary to the console.


In [ ]:
# Define the height and width of the images to be processed.
# These dimensions should be adjusted based on the requirements of your model 
# and the characteristics of your dataset. All images will be resized to this size.
img_height, img_width = 150, 150  # Image dimensions (height, width)

# Set the batch size for processing images during training and validation.
# The batch size determines how many images will be fed into the model at once 
# for training or evaluation, impacting the speed and memory usage during training.
batch_size = 8  # Number of images to process in a batch


In [ ]:
# Enhanced ImageDataGenerator for the training dataset with tuned augmentation parameters.
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,               # Normalize pixel values for improved model training stability.
    rotation_range=15,                 # Slightly reduce rotation range for a more realistic augmentation.
    width_shift_range=0.15,            # Limit horizontal shifts to 15% to avoid excessive transformations.
    height_shift_range=0.15,           # Limit vertical shifts to 15%.
    shear_range=0.1,                   # Reduce shear transformation to prevent image distortion.
    zoom_range=[0.9, 1.1],             # Slight zoom in/out to generalize scale invariance.
    horizontal_flip=True,              # Enable horizontal flips (if applicable to your dataset).
    brightness_range=[0.8, 1.2],       # Randomly adjust brightness for lighting variations.
    fill_mode='nearest'                # Fill in new pixels with nearest values to avoid artifacts.
)

# Validation ImageDataGenerator with only rescaling for more reliable validation performance.
validation_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0
)


In [ ]:
# Create a data generator for the training set using the ImageDataGenerator's flow_from_directory method
train_generator = train_datagen.flow_from_directory(
    train_data_dir,  # Directory containing the training data
    target_size=(img_height, img_width),  # Resize the images to the target size (img_height, img_width)
    batch_size=batch_size,  # Number of images to process in one batch
    class_mode='categorical',  # For multi-class classification (one-hot encoded labels)
)

# Create a data generator for the validation set, similarly to the training set
validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,  # Directory containing the validation data
    target_size=(img_height, img_width),  # Resize the images to the target size
    batch_size=batch_size,  # Number of images to process in one batch
    class_mode='categorical',  # For multi-class classification (one-hot encoded labels)
)


In [ ]:
print(train_generator.class_indices)

label_dict = {v: k for k, v in train_generator.class_indices.items()}
print(label_dict)

In [ ]:
# Define the ResNet50 model with custom input size and no top layer
resnet_base = ResNet50(
    weights='imagenet',            # Load pre-trained weights from ImageNet
    include_top=False,             # Exclude the fully connected layer at the top
    input_shape=(img_height, img_width, 3)  # Input image dimensions
)

In [ ]:
# Freeze the ResNet base model layers
resnet_base.trainable = False

In [ ]:
x = resnet_base.output
x = layers.GlobalAveragePooling2D()(x)

# First fully connected layer with Batch Normalization and L2 regularization
x = layers.Dense(516, activation=None, kernel_regularizer='l2')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

x = layers.Dropout(0.5)(x)  # Higher dropout rate for better regularization

# # Second fully connected layer with Batch Normalization and L2 regularization
# x = layers.Dense(516, activation=None, kernel_regularizer='l2')(x)
# x = layers.BatchNormalization()(x)
# x = layers.ReLU()(x)

# Third fully connected layer with Batch Normalization and L2 regularization
x = layers.Dense(256, activation=None, kernel_regularizer='l2')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

x = layers.Dropout(0.5)(x)  # Higher dropout rate for better regularization

# # Fourth fully connected layer with Batch Normalization and L2 regularization
# x = layers.Dense(128, activation=None, kernel_regularizer='l2')(x)
# x = layers.BatchNormalization()(x)
# x = layers.ReLU()(x)

output_layer = layers.Dense(train_generator.num_classes, activation='softmax')(x)


In [ ]:
# Create the final model
model = models.Model(inputs=resnet_base.input, outputs=output_layer)

In [ ]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6),  # Use the Adam optimizer for training, which is efficient and widely used.
              loss='categorical_crossentropy',  # Specify categorical crossentropy as the loss function for multi-class classification.
              metrics=['accuracy'])  # Track accuracy as the performance metric during training and evaluation.

In [ ]:
# Model summary
model.summary()

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Learning Rate Reduction Callback: Reduces learning rate when validation loss has stopped improving.
# Monitors 'val_loss' and reduces the learning rate by a smaller factor if no improvement for a specific number of epochs.
# A minimum learning rate and cooldown period are included to prevent over-reduction and frequent adjustments.

lr_reduction = ReduceLROnPlateau(
    monitor='val_accuracy',      # Metric to monitor
    factor=0.7,              # Reduce the learning rate by half (new_lr = lr * factor)
    patience=2,              # Reduce the learning rate after 4 epochs of no improvement
    verbose=1,               # Print a message when reducing the learning rate
    min_lr=1e-4,             # Minimum learning rate to avoid becoming too low
    cooldown=2               # Wait 2 epochs after a reduction before resuming monitoring
)

In [ ]:
# Custom callback to stop training at 92% accuracy
class StopAtAccuracy(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):  # This method is called at the end of each epoch during training.
        if logs["accuracy"] >= 0.90:  # Check if the accuracy for the epoch has reached or exceeded 98%.
            self.model.stop_training = True  # Stop the training process if the condition is met.


In [ ]:
epochs = 100

# Train the model using the fit method, which adjusts the model's weights based on training data
history = model.fit(
    train_generator,  # The training data generator providing batches of training images and labels
    validation_data=validation_generator,  # The validation data generator for evaluating the model during training
    epochs=epochs,  # The number of complete passes through the training dataset
    callbacks=[StopAtAccuracy(), lr_reduction]  # Include callbacks for stopping the training early and reducing the learning rate
)


In [ ]:
import matplotlib.pyplot as plt  # Import the matplotlib library for plotting.

# Create a figure with a specified size for the plots.
plt.figure(figsize=(12, 4))

# Create the first subplot for accuracy values.
plt.subplot(1, 2, 1)  # 1 row, 2 columns, first subplot
plt.plot(history.history['accuracy'], label='Train Accuracy')  # Plot training accuracy.
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')  # Plot validation accuracy.
plt.title('Model Accuracy')  # Title of the accuracy plot.
plt.ylabel('Accuracy')  # Label for the y-axis.
plt.xlabel('Epoch')  # Label for the x-axis.
plt.legend(loc='upper left')  # Add a legend to the plot.

# Create the second subplot for loss values.
plt.subplot(1, 2, 2)  # 1 row, 2 columns, second subplot
plt.plot(history.history['loss'], label='Train Loss')  # Plot training loss.
plt.plot(history.history['val_loss'], label='Validation Loss')  # Plot validation loss.
plt.title('Model Loss')  # Title of the loss plot.
plt.ylabel('Loss')  # Label for the y-axis.
plt.xlabel('Epoch')  # Label for the x-axis.
plt.legend(loc='upper left')  # Add a legend to the plot.

plt.tight_layout()  # Adjust layout to prevent overlap of subplots.
plt.show()  # Display the plots.


In [ ]:
# Save the model after training in the Keras format
model.save('sign_language_detection_model.keras')  # Save the entire model architecture, weights, and training configuration to a file named 'sign_language_detection_model.keras'.


In [ ]:
# Generate predictions for the validation set
val_predictions = model.predict(validation_generator)
y_pred = np.argmax(val_predictions, axis=1)
y_true = validation_generator.classes

# Calculate and display metrics
print("Accuracy:", accuracy_score(y_true, y_pred))
print("F1 Score:", f1_score(y_true, y_pred, average='weighted'))
print("Precision:", precision_score(y_true, y_pred, average='weighted'))
print("Recall:", recall_score(y_true, y_pred, average='weighted'))

# Detailed classification report
print("\nClassification Report:\n", classification_report(y_true, y_pred, target_names=list(label_dict.values())))

# Plot confusion matrix
ConfusionMatrixDisplay.from_predictions(y_true, y_pred, display_labels=list(label_dict.values()), cmap='Blues')
plt.title("Confusion Matrix")
plt.show()